In [1]:

#1. 음악 데이터 파일 로드에 필요한 라이브러리 
import IPython.display
import pandas as pd
import numpy as np

# 음악 분석 라이브러리
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [2]:
#2. 음악 정보 파일 불러오기
# music_inform.csv 

music_inform=pd.read_csv('music_inform.csv')

display(music_inform)

,Music,Genre,ID,Format,Path
0,Contact High - RKVC,RnBSoul_Calm,R_C1,mp3,RnBSoul_Calm/R_C1.mp3
1,QuangerineCream - Noir Et Blanc Vie,RnBSoul_Calm,R_C2,mp3,RnBSoul_Calm/R_C2.mp3
2,TrueArtRealAffectionPart4 - Noir Et Blanc Vie,RnBSoul_Calm,R_C3,mp3,RnBSoul_Calm/R_C3.mp3
3,Wallflowers - Bad Snacks,RnBSoul_Calm,R_C4,mp3,RnBSoul_Calm/R_C4.mp3
4,Where To_ - Birocratic,RnBSoul_Calm,R_C5,mp3,RnBSoul_Calm/R_C5.mp3
5,Goji Berry Beat - Nana Kwabena,Hiphop_Funky,H_F1,mp3,Hiphop_Funky/H_F1.mp3
6,Madam Wahala Beat - Nana Kwabena,Hiphop_Funky,H_F2,mp3,Hiphop_Funky/H_F2.mp3
7,Pretty Boy - DJ Freedem,Hiphop_Funky,H_F3,mp3,Hiphop_Funky/H_F3.mp3
8,Road Tripzzz - Ofshane,Hiphop_Funky,H_F4,mp3,Hiphop_Funky/H_F4.mp3
9,Twisted Bandits All Around Me - NoMBe,Hiphop_Funky,H_F5,mp3,Hiphop_Funky/H_F5.mp3


In [3]:
#3. 분석에 필요한 열만 추출 : music_data
#ID, Path

music_data=music_inform[['ID','Path']]

display(music_data)

,ID,Path
0,R_C1,RnBSoul_Calm/R_C1.mp3
1,R_C2,RnBSoul_Calm/R_C2.mp3
2,R_C3,RnBSoul_Calm/R_C3.mp3
3,R_C4,RnBSoul_Calm/R_C4.mp3
4,R_C5,RnBSoul_Calm/R_C5.mp3
5,H_F1,Hiphop_Funky/H_F1.mp3
6,H_F2,Hiphop_Funky/H_F2.mp3
7,H_F3,Hiphop_Funky/H_F3.mp3
8,H_F4,Hiphop_Funky/H_F4.mp3
9,H_F5,Hiphop_Funky/H_F5.mp3


In [4]:
#4. 모든 곡들의 spectral_rolloff=[]저장 
n=len(music_data) #데이터 수
spectral_rolloff_max=[]
spectral_rolloff_min=[]

for i in range(n):
    music_name=music_data.loc[i]
    music_path=music_name[1]
    
    y,sr=librosa.load(music_path,sr=44100,duration=10)
    IPython.display.Audio(data=y,rate=sr)


    #spectral_rolloff_max 추출
    r_o_max=librosa.feature.spectral_rolloff(y,sr=sr,roll_percent=0.99)
    r_o_min=librosa.feature.spectral_rolloff(y,sr=sr,roll_percent=0.01)
    
    #spectral_centroid.append(s_c)
    spectral_rolloff_max.append(r_o_max)
    spectral_rolloff_min.append(r_o_min)
    

frame_num=len(spectral_rolloff_max[0][0]) #프레임 수

c:\users\yeongeun\appdata\local\programs\python\python38-32\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [5]:
#5. DTW함수
def DTW(a,b,length):
    #5-1. dtw를 구하기 위한 행렬
    dtw=[[float('inf') for i in range(length)] for j in range(length)]
    
    #5-2. dtw 첫행값 구하기
    dtw[0][0]=abs(a[0]-b[0])
    
    for dj in range(1,length):
        dtw[0][dj]=dtw[0][dj-1]+abs(a[0]-b[dj])
    
    #5-3. dtw 첫열값 구하기
    for di in range(1,length):
        dtw[di][0]=dtw[di-1][0]+abs(a[di]-b[0])
        
    #5-4. dtw 채우기
    for di in range(1,length):
        for dj in range(1,length):
            min_dtw=min(min(dtw[di-1][dj-1],dtw[di-1][dj]),dtw[di][dj-1])
            
            dtw[di][dj]=min_dtw+abs(a[di]-b[dj])
            
    #5-5. 최종 dtw 유사거리
    return dtw[length-1][length-1]
    
    
    

In [6]:
#6. spectral_rolloff_max 유사 구해보기
s_c_similarity_max=[[0 for i in range(n)] for j in range(n)]

for i in range(n):
    for j in range(i+1,n):
        s_c_similarity_max[i][j]=DTW(spectral_rolloff_max[i][0],spectral_rolloff_max[j][0],frame_num)
        s_c_similarity_max[j][i]=s_c_similarity_max[i][j]
        
        #현재 진행상황 파악하기 위한 print
        print('[',i,']','[',j,']의 s_c_similarity_max 유사도 : ', s_c_similarity_max[i][j])


[ 0 ] [ 1 ]의 s_c_similarity_max 유사도 :  1710834.521484375
[ 0 ] [ 2 ]의 s_c_similarity_max 유사도 :  3202461.03515625
[ 0 ] [ 3 ]의 s_c_similarity_max 유사도 :  3532844.970703125
[ 0 ] [ 4 ]의 s_c_similarity_max 유사도 :  2863700.68359375
[ 0 ] [ 5 ]의 s_c_similarity_max 유사도 :  2791435.25390625
[ 0 ] [ 6 ]의 s_c_similarity_max 유사도 :  2314281.005859375
[ 0 ] [ 7 ]의 s_c_similarity_max 유사도 :  1361932.03125
[ 0 ] [ 8 ]의 s_c_similarity_max 유사도 :  3256423.2421875
[ 0 ] [ 9 ]의 s_c_similarity_max 유사도 :  3614778.80859375
[ 0 ] [ 10 ]의 s_c_similarity_max 유사도 :  2718114.697265625
[ 0 ] [ 11 ]의 s_c_similarity_max 유사도 :  2237062.939453125
[ 0 ] [ 12 ]의 s_c_similarity_max 유사도 :  1865356.787109375
[ 0 ] [ 13 ]의 s_c_similarity_max 유사도 :  2512558.740234375
[ 0 ] [ 14 ]의 s_c_similarity_max 유사도 :  3454421.044921875
[ 1 ] [ 2 ]의 s_c_similarity_max 유사도 :  2721129.345703125
[ 1 ] [ 3 ]의 s_c_similarity_max 유사도 :  1897419.7265625
[ 1 ] [ 4 ]의 s_c_similarity_max 유사도 :  3865812.890625
[ 1 ] [ 5 ]의 s_c_similarity_max 유사도 :  19

In [7]:
#7. s_c_similarity_min 유사 구해보기
s_c_similarity_min=[[0 for i in range(n)] for j in range(n)]

for i in range(n):
    for j in range(i+1,n):
        s_c_similarity_min[i][j]=DTW(spectral_rolloff_min[i][0],spectral_rolloff_min[j][0],frame_num)
        s_c_similarity_min[j][i]=s_c_similarity_min[i][j]
        
        #현재 진행상황 파악하기 위한 print
        print('[',i,']','[',j,']의 s_c_similarity_min 유사도 : ', s_c_similarity_min[i][j])


[ 0 ] [ 1 ]의 s_c_similarity_min 유사도 :  61994.091796875
[ 0 ] [ 2 ]의 s_c_similarity_min 유사도 :  42528.076171875
[ 0 ] [ 3 ]의 s_c_similarity_min 유사도 :  24009.521484375
[ 0 ] [ 4 ]의 s_c_similarity_min 유사도 :  93690.966796875
[ 0 ] [ 5 ]의 s_c_similarity_min 유사도 :  47200.78125
[ 0 ] [ 6 ]의 s_c_similarity_min 유사도 :  5813.96484375
[ 0 ] [ 7 ]의 s_c_similarity_min 유사도 :  17420.361328125
[ 0 ] [ 8 ]의 s_c_similarity_min 유사도 :  68690.91796875
[ 0 ] [ 9 ]의 s_c_similarity_min 유사도 :  47222.314453125
[ 0 ] [ 10 ]의 s_c_similarity_min 유사도 :  79608.251953125
[ 0 ] [ 11 ]의 s_c_similarity_min 유사도 :  69789.111328125
[ 0 ] [ 12 ]의 s_c_similarity_min 유사도 :  8354.8828125
[ 0 ] [ 13 ]의 s_c_similarity_min 유사도 :  95370.556640625
[ 0 ] [ 14 ]의 s_c_similarity_min 유사도 :  2670.1171875
[ 1 ] [ 2 ]의 s_c_similarity_min 유사도 :  13608.984375
[ 1 ] [ 3 ]의 s_c_similarity_min 유사도 :  28531.494140625
[ 1 ] [ 4 ]의 s_c_similarity_min 유사도 :  26593.505859375
[ 1 ] [ 5 ]의 s_c_similarity_min 유사도 :  16063.76953125
[ 1 ] [ 6 ]의 s_c_simil

In [8]:
#7. 3곡정도씩 유사곡 리스트 만들어보기 
#r_num : 유사곡 수 : 3
#music_num : 자기 자신 번호
#s: s_c_similarity[music_num]

def recommendation(music_num,s,r_num):
    s_list=[]
    
    for i in range(len(s)):
        if music_num==i:
            continue
        s_list.append([s[i],i])
        
    s_list=sorted(s_list)
    
    return [s_list[i][1] for i in range(r_num)]


r_num=5
for i in range(n):
    print(i,'곡의 spectral_rolloff 유사 곡 : ',recommendation(i,s_c_similarity_max[i],r_num),recommendation(i,s_c_similarity_min[i],r_num))
    
    

0 곡의 spectral_rolloff 유사 곡 :  [7, 1, 12, 11, 6] [14, 6, 12, 7, 3]
1 곡의 spectral_rolloff 유사 곡 :  [0, 6, 3, 5, 7] [2, 5, 9, 10, 12]
2 곡의 spectral_rolloff 유사 곡 :  [9, 10, 3, 5, 14] [9, 5, 1, 7, 10]
3 곡의 spectral_rolloff 유사 곡 :  [5, 9, 2, 10, 1] [12, 7, 14, 2, 9]
4 곡의 spectral_rolloff 유사 곡 :  [11, 8, 7, 0, 13] [10, 5, 2, 1, 13]
5 곡의 spectral_rolloff 유사 곡 :  [9, 3, 10, 14, 2] [2, 9, 1, 3, 10]
6 곡의 spectral_rolloff 유사 곡 :  [1, 12, 7, 0, 3] [0, 14, 12, 7, 3]
7 곡의 spectral_rolloff 유사 곡 :  [0, 1, 12, 6, 10] [12, 14, 3, 2, 0]
8 곡의 spectral_rolloff 유사 곡 :  [4, 11, 7, 13, 0] [2, 3, 10, 12, 4]
9 곡의 spectral_rolloff 유사 곡 :  [5, 2, 3, 10, 14] [2, 5, 3, 7, 1]
10 곡의 spectral_rolloff 유사 곡 :  [2, 5, 9, 3, 7] [2, 4, 5, 1, 3]
11 곡의 spectral_rolloff 유사 곡 :  [4, 8, 0, 7, 1] [13, 10, 1, 4, 2]
12 곡의 spectral_rolloff 유사 곡 :  [0, 6, 7, 1, 13] [14, 0, 7, 3, 6]
13 곡의 spectral_rolloff 유사 곡 :  [0, 7, 14, 1, 12] [4, 10, 1, 2, 5]
14 곡의 spectral_rolloff 유사 곡 :  [5, 9, 3, 1, 2] [0, 12, 6, 7, 3]


In [11]:
#7. 곡별 추천 리스트 파일로 저장하기

#recomsize: 추천 리스트
#header 생성
recomsize=['recom'+str(i) for i in range(1,r_num+1)]
header=['music_id']+recomsize

recom_list=[]

for i in range(n):
    recom_list.append([i]+recommendation(i,s_c_similarity_max[i],r_num))
    
recom_df=pd.DataFrame(recom_list,columns=header)
display(recom_df)

#csv파일로 저장
recom_df.to_csv('spectral_rolloff_max_recommendation.csv',header=True,index=False)

,music_id,recom1,recom2,recom3,recom4,recom5
0,0,7,1,12,11,6
1,1,0,6,3,5,7
2,2,9,10,3,5,14
3,3,5,9,2,10,1
4,4,11,8,7,0,13
5,5,9,3,10,14,2
6,6,1,12,7,0,3
7,7,0,1,12,6,10
8,8,4,11,7,13,0
9,9,5,2,3,10,14


In [12]:
recom_list=[]

for i in range(n):
    recom_list.append([i]+recommendation(i,s_c_similarity_min[i],r_num))
    
recom_df=pd.DataFrame(recom_list,columns=header)
display(recom_df)

#csv파일로 저장
recom_df.to_csv('spectral_rolloff_min_recommendation.csv',header=True,index=False)

,music_id,recom1,recom2,recom3,recom4,recom5
0,0,14,6,12,7,3
1,1,2,5,9,10,12
2,2,9,5,1,7,10
3,3,12,7,14,2,9
4,4,10,5,2,1,13
5,5,2,9,1,3,10
6,6,0,14,12,7,3
7,7,12,14,3,2,0
8,8,2,3,10,12,4
9,9,2,5,3,7,1
